In [ ]:
## Pull all of  the annotations for the easy study

In [1]:
import girder_client
##  Connect to ISIC in order to pull ISIC annotation Data
BASE_URL = 'https://isic-archive.com/api/v1'
gc = girder_client.GirderClient(apiUrl=BASE_URL)
gc.authenticate(interactive=True)

Login or email: dagutman@gmail.com
Password for dagutman@gmail.com: ········


{'_accessLevel': 2,
 '_id': '5436c6e7bae4780a676c8f93',
 '_modelType': 'user',
 'admin': False,
 'created': '2014-10-09T17:33:27.605000+00:00',
 'email': 'dagutman@gmail.com',
 'emailVerified': False,
 'firstName': 'David',
 'gravatar_baseUrl': 'https://www.gravatar.com/avatar/287ea7e17f6939c4677e5e1662291644?d=identicon',
 'groupInvites': [],
 'groups': ['57e2e41e9fc3c17e95ca4c78',
  '575183509fc3c111cbe228cd',
  '576ad77f9fc3c178e90c9442'],
 'lastName': 'Gutman',
 'login': 'dagutman',
 'otp': False,
 'permissions': {'acceptTerms': True,
  'adminStudy': True,
  'createDataset': True,
  'reviewDataset': False,
  'segmentationSkill': None},
 'public': False,
 'size': 7182528957,
 'status': 'enabled'}

In [4]:
WhiteListedStudySet = ['EASY_STUDY_1','EASY_STUDY_2','EASY_STUDY_3','EASY_STUDY_4']

studyList = gc.get('study')
for s in studyList:
    
    studyName = s['name']
    
    if (studyName in WhiteListedStudySet): ## Pull data only for the relevant studies
        studyDetails = gc.get('study/%s' % s['_id'])  # Get the list of images for that study
#         print(studyDetails)
#         break
        studyLastUpdated = studyDetails['updated']
        featureSet = studyDetails['features']  ## note camel case vs non camelcase
        featureSetName = "EASY_STUDY" ##featureSet['name']
        featureSetID = "NA" ### featureSet['_id']
        studyImageList = studyDetails['images']
        
        #curFeatureSetData = gc.get('featureset/%s' % featureSetID)
        
        curStudyRaterData = {}
        for si in studyImageList:
            print(si['name'])
            curImageAnnotations = gc.get('annotation?studyId=%s&imageId=%s' % ( s['_id'], si['_id'])) ### Each image can then have many annotations
            
            curStudyRaterData[si['name']] =  AnalyzeImageAnnotations(gc,curImageAnnotations) 
            #pprint.pprint(raterData)
        AllStudyRaterData[studyName] = { "FeatureSetId": studyDetails["featureset"]["_id"],
                                        "MarkupData": curStudyRaterData, "fullFeatureSet" : curFeatureSetData
                                        }


ISIC_0016081
ISIC_0016092
ISIC_0016127
ISIC_0016131


KeyboardInterrupt: 

In [37]:
curImageAnnotations = gc.get('annotation?studyId=%s&imageId=%s' % ( s['_id'], si['_id'])) 
AnalyzeImageAnnotations(gc,curImageAnnotations)

9
dict_keys(['_id', '_modelType', 'image', 'log', 'markups', 'responses', 'startTime', 'state', 'status', 'stopTime', 'studyId', 'user'])
{'_id': '5c0564771165972a676b3605',
 '_modelType': 'annotation',
 'image': {'_id': '58b0a374d831137d0a3884c5',
           'name': 'ISIC_0016131',
           'updated': '2017-02-24T21:34:40.938000+00:00'},
 'log': [{'detail': {},
          'time': '2018-12-17T02:11:27.758Z',
          'type': 'annotation_start'},
         {'detail': {'featureId': 'Vessels : Linear irregular'},
          'time': '2018-12-17T02:11:44.201Z',
          'type': 'markup_select'},
         {'detail': {'featureId': 'Vessels : Linear irregular'},
          'time': '2018-12-17T02:12:25.329Z',
          'type': 'markup_set'},
         {'detail': {'featureId': 'Vessels : Dotted'},
          'time': '2018-12-17T02:12:25.343Z',
          'type': 'markup_select'},
         {'detail': {'featureId': 'Vessels : Dotted'},
          'time': '2018-12-17T02:12:42.528Z',
          'type': '

dict_keys(['_id', '_modelType', 'image', 'log', 'markups', 'responses', 'startTime', 'state', 'status', 'stopTime', 'studyId', 'user'])
{'_id': '5c1433234cb5374b8c139b6e',
 '_modelType': 'annotation',
 'image': {'_id': '58b0a374d831137d0a3884c5',
           'name': 'ISIC_0016131',
           'updated': '2017-02-24T21:34:40.938000+00:00'},
 'log': [{'detail': {},
          'time': '2018-12-20T10:54:02.600Z',
          'type': 'annotation_start'},
         {'detail': {'featureId': 'Dots : Irregular'},
          'time': '2018-12-20T10:54:12.304Z',
          'type': 'markup_select'},
         {'detail': {'featureId': 'Dots : Irregular'},
          'time': '2018-12-20T10:54:40.846Z',
          'type': 'markup_set'},
         {'detail': {'featureId': 'Vessels : Linear irregular'},
          'time': '2018-12-20T10:54:40.852Z',
          'type': 'markup_select'},
         {'detail': {'featureId': 'Vessels : Linear irregular'},
          'time': '2018-12-20T10:54:49.205Z',
          'type': 'ma

             'Vessels : Polymorphous': True},
 'responses': {'Classify the Lesion as Benign or Malignant': 'Malignant',
               'Classify the Lesion as Organized or Disorganized': 'Disorganized',
               'What is your level of confidence?': 'Very Confident'},
 'startTime': '2019-05-25T16:17:25.533000+00:00',
 'state': 'complete',
 'status': 'ok',
 'stopTime': '2019-05-25T16:27:48.085000+00:00',
 'studyId': '5bf578441165972a676b19c9',
 'user': {'_id': '5c1435124cb5374b8c139be8',
          'firstName': 'Susana',
          'lastName': 'Puig',
          'login': 'susipuig',
          'name': 'User 57HO'}}


{'raters': {}}

In [ ]:
### need to look into the modelType as well

In [36]:
import pprint as pprint
def AnalyzeImageAnnotations(gc, curImageAnnotations):
    """An image for a particular study can have one or more annotations, that need ot be cleaned up and
    combined into a single ARRAY for visualization, this function takes a pointer to the girer instance
    with the annotations and I will likely be adding additional params as this iterates
    The curImageAnnotations contains a list of dictionaries containing all the _ids for the annotations"""
    ### First get all the annotations, may debate filtering by user, and also add a 
    ### AggreGate user
    raterData = {}
    
    for cia in curImageAnnotations:
#         print cia
        cuad = gc.get('annotation/%s' % cia['_id'] ) ## Current User Annotation Data
        
        if (cuad['state']== 'complete'):
            print (cuad.keys())
            pprint.pprint(cuad)

#         try:
#         cuadState = cuad['state']  ## Means its completed or not completed
#         cuadUserName =  cuad['user']['login']  
# #             cuadUserAnon = cuad['user']
#         cuadAnnotationData = cuad['annotations']
#         raterData[cuadUserName] = {}
#         raterData[cuadUserName]['meta'] = {}
#         raterData[cuadUserName]['meta']['annotations'] =  {}
            
#         stop = dateutil.parser.parse(cuad['startTime'] )
#         start = dateutil.parser.parse(cuad['stopTime'] )
#         markupTime  = "%s" % (stop-start)
            
#         for ftr in cuadAnnotationData.keys():
#             ### Not all the features are necessarily numeric
#             try:
#                 featureSum =  sum(cuadAnnotationData[ftr])
#                 ### Probably just check and see if it's a string or an arrary.. would be easier
#                 ### added the[0:5] during debugging
#                 if featureSum > 0:
#                     raterData[cuadUserName]['meta']['annotations'][ftr] =  cuadAnnotationData[ftr]

#                     raterData[cuadUserName]['meta']['markupTime']  = markupTime
#                     raterData[cuadUserName]['meta']['startTime']  = start.isoformat()
#                     raterData[cuadUserName]['meta']['status']  = "StatusOKGoeshere"

#                     raterData[cuadUserName]['updated']  = start.isoformat()
#                     raterData[cuadUserName]['name']  = "ImageNAMEGOES HERE"

# #                     else:
# #                         pass
# #                         print "Blank feature detected"
#             except:
#                 pass
                
#         except:
#             pass
            #print "Incomplete data for",cuad
    return {"raters": raterData }
